In [3]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, RNN, Lambda, Concatenate, Reshape, Add
from keras.layers.merge import Multiply
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy
from keras.models import Model
from keras.models import load_model
from keras.models import model_from_json
import keras.backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
def open_and_load(string):
    with open('models/'+string+'.json', 'r') as f:
        model = model_from_json(f.read())
    model.load_weights('models/'+string+'.h5')   
    return model

In [5]:
model_2d = open_and_load('model_2d')

In [62]:
def gen(batch_size=128):
    while True:
        a = np.zeros((batch_size,4)) #x, v, b1, b2
        a[:,1] = np.random.uniform(0,0.1,size=batch_size)
        a[:,2] = np.random.uniform(1,-10,size=batch_size)
        a[:,3] = np.random.uniform(1, 10,size=batch_size)
        i = np.random.choice([0,1], size=batch_size) #isb1
        b = a[:,2]*i + a[:,3]*(1-i)
        a[:,1] *= np.sign(1-i-0.5) 
        a[:,0] = np.random.uniform(b, b-10*a[:,1])
        
        isBounce = ( np.abs(a[:,0] + a[:,1]) - np.abs(b)) > 0
        
        v = a[:,1] * np.sign(1-isBounce-0.5) #new speed
        x = (a[:,0] + a[:,1]) * (1-isBounce) + (2*b-a[:,1]-a[:,0])*isBounce
        b_newIsb1 = ((x-a[:,2])/(a[:,3]-a[:,2])) < 0.5
        b_new = a[:,2]*b_newIsb1 + a[:,3]*(1-b_newIsb1)
        y = np.zeros((batch_size,2))
        y[:,0] = x
        y[:,1] = v
        yield a, y     
        
def gen_2d(batch_size=128):
    while True:
        a = np.zeros((batch_size,8)) #x, v, b1, b2
        y = np.zeros((batch_size,4))
        a[:,:4], y[:,:2] = next(gen(batch_size=batch_size))
        a[:,4:], y[:,2:] = next(gen(batch_size=batch_size))
        print(a.shape, y.shape)
        yield a, y
        
def save_model(model, model_str):
    """Saves model with weights."""
    model_json = model.to_json()
    with open('models/'+model_str+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('models/'+model_str+".h5")
    print("Saved model to disk, name: ", model_str)

In [13]:
next(gen_2d(batch_size=1))

(1, 8) (1, 4)


(array([[-5.15478406, -0.01945439, -5.28971697,  3.57491543,  0.17876718,
         -0.05096104,  0.16473999,  9.38289792]]),
 array([[-5.17423845, -0.01945439,  0.12780615, -0.05096104]]))

In [21]:
df = pd.read_csv('df.csv')
df.x_vel /= 200
df.y_vel /= 200

In [29]:
df.shape

(9999, 4)

In [34]:
df.max()

x        0.863201
y        0.817080
x_vel    0.008867
y_vel    0.001176
dtype: float64

In [57]:
X = np.zeros((9998,8))
X[:,[0,1,4,5]] = df.iloc[:-1].to_numpy()[:,[0,2,1,3]] #ball coordinates and speeds
X[:,[2,3,6,7]] = [-0.865493, -0.816913, 0.863201, 0.817080] #borders
Y = df.iloc[1:].to_numpy()[:,[0,2,1,3]]
P = model_2d.predict(X)

In [61]:
mse(Y,P)

2.3690914647027288e-05

In [58]:
pd.DataFrame(X[1:3])

,0,1,2,3,4,5,6,7
0,-0.009217,-0.009217,-0.865493,-0.816913,-0.000907,-0.000907,0.863201,0.81708
1,-0.018435,-0.009218,-0.865493,-0.816913,-0.001813,-0.000907,0.863201,0.81708


In [59]:
pd.DataFrame(Y[1:3])

,0,1,2,3
0,-0.018435,-0.009218,-0.001813,-0.000907
1,-0.027653,-0.009218,-0.002720,-0.000907


In [60]:
pd.DataFrame(P[1:3])

,0,1,2,3
0,-0.010920,0.009217,-0.001195,0.000907
1,-0.020138,0.009218,-0.002101,0.000907


In [54]:
model_2d.compile(loss=mean_squared_error,
              optimizer='adam')

In [56]:
model_2d.fit(X,Y, epochs=100)

Epoch 1/100
9998/9998 [==============================] - 1s 131us/step - loss: 1.1620
Epoch 2/100
9998/9998 [==============================] - 1s 131us/step - loss: 0.8024
Epoch 3/100
9998/9998 [==============================] - 1s 131us/step - loss: 0.5330
Epoch 4/100
9998/9998 [==============================] - 1s 133us/step - loss: 0.3377
Epoch 5/100
9998/9998 [==============================] - 1s 132us/step - loss: 0.2019
Epoch 6/100
9998/9998 [==============================] - 1s 131us/step - loss: 0.1125
Epoch 7/100
9998/9998 [==============================] - 1s 132us/step - loss: 0.0575
Epoch 8/100
9998/9998 [==============================] - 1s 131us/step - loss: 0.0265
Epoch 9/100
9998/9998 [==============================] - 1s 132us/step - loss: 0.0107
Epoch 10/100
9998/9998 [==============================] - 1s 132us/step - loss: 0.0038
Epoch 11/100
9998/9998 [==============================] - 1s 132us/step - loss: 0.0011
Epoch 12/100
9998/9998 [============================

KeyboardInterrupt: 

In [64]:
# save_model(model_2d, 'model_2d')